In [5]:
!pip install GEOparse
!pip install combat
!pip install scipy
import GEOparse
#downloading the module that will allow us to load data from Genetic Omnibus
import pandas as pd
#downloading pandas for making arrays and accessing data
import numpy as np
#pretty similar to pandas
from sklearn.preprocessing import StandardScaler
#used in pre-processing for helping standardize data with multiple datasets
from sklearn.impute import SimpleImputer
#used in pre-processing for replacing missing values
from combat.pycombat import pycombat
#batch effects

#Feature selection packages
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.feature_selection import SelectKBest, chi2
import pandas as pd
from scipy.stats import ttest_ind

#Machine Learning Algorithms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

  Preparing metadata (setup.py) ... done
  Created wheel for combat: filename=combat-0.3.3-py3-none-any.whl size=36777 sha256=f488e0b7bd682d20f734fbea08a867c8d8234aa8e73b53dcd32ccff80ad0e401
  Stored in directory: /root/.cache/pip/wheels/0d/e4/46/5c7b89d38f5160cf8f2fdda0a52c93439b6e7d9d661e8336f4
Successfully built combat


In [6]:
GSE3307 = GEOparse.get_GEO(geo="GSE3307", destdir="./") # remove the [0]
GSE6011 = GEOparse.get_GEO(geo="GSE6011", destdir="./") # remove the [0]
#accessing THESE specific datasets
#destdir ./ --> saves to working directory for faster processing

# To access specific data within the GSE object, explore its attributes and methods.
# For example, to get the metadata:
print(GSE3307.metadata)

# To get a list of samples:
print(GSE3307.gsms)
#This attribute contains a dictionary of GSM (Sample) objects. Each key in the dictionary is a GSM accession number, and the corresponding value is a GSM object that holds information about the individual sample, such as metadata, expression data, and annotations.

16-Jul-2024 04:57:29 DEBUG utils - Directory ./ already exists. Skipping.
DEBUG:GEOparse:Directory ./ already exists. Skipping.
16-Jul-2024 04:57:29 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE3307_family.soft.gz to ./GSE3307_family.soft.gz
INFO:GEOparse:Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE3307_family.soft.gz to ./GSE3307_family.soft.gz
100%|██████████| 62.8M/62.8M [00:04<00:00, 13.3MB/s]
16-Jul-2024 04:57:35 DEBUG downloader - Size validation passed
DEBUG:GEOparse:Size validation passed
16-Jul-2024 04:57:35 DEBUG downloader - Moving /tmp/tmpuy0j4531 to /content/GSE3307_family.soft.gz
DEBUG:GEOparse:Moving /tmp/tmpuy0j4531 to /content/GSE3307_family.soft.gz
16-Jul-2024 04:57:36 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE3307_family.soft.gz
DEBUG:GEOparse:Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE330

{'title': ['Comparative profiling in 13 muscle disease groups'], 'geo_accession': ['GSE3307'], 'status': ['Public on Oct 14 2005'], 'submission_date': ['Sep 14 2005'], 'last_update_date': ['Oct 01 2019'], 'pubmed_id': ['16478798', '25313409'], 'summary': ['Summary:   Genetic disorders of muscle cause muscular dystrophy, and are some of the most common inborn errors of metabolism. Muscle also rapidly remodels in response to training and innervation. Muscle weakness and wasting is important in such conditions as aging, critical care medicine, space flight, and diabetes. Finally, muscle can also be used to investigate systemic defects, and the compensatory mechansisms invoked by cells to overcome biochemical and genetic abnormalities. Here, we provide a 13 group data set for comparative profiling of human skeletal muscle. Groups studied are: Normal human skeletal muscle, Acute quadriplegic myopathy (AQM; critical care myopathy), Juvenile dermatomyositis (JDM), Amyotophic lateral sclerosis

In [7]:
#Metadata
print(GSE3307.metadata)
print(GSE6011.metadata)

{'title': ['Comparative profiling in 13 muscle disease groups'], 'geo_accession': ['GSE3307'], 'status': ['Public on Oct 14 2005'], 'submission_date': ['Sep 14 2005'], 'last_update_date': ['Oct 01 2019'], 'pubmed_id': ['16478798', '25313409'], 'summary': ['Summary:   Genetic disorders of muscle cause muscular dystrophy, and are some of the most common inborn errors of metabolism. Muscle also rapidly remodels in response to training and innervation. Muscle weakness and wasting is important in such conditions as aging, critical care medicine, space flight, and diabetes. Finally, muscle can also be used to investigate systemic defects, and the compensatory mechansisms invoked by cells to overcome biochemical and genetic abnormalities. Here, we provide a 13 group data set for comparative profiling of human skeletal muscle. Groups studied are: Normal human skeletal muscle, Acute quadriplegic myopathy (AQM; critical care myopathy), Juvenile dermatomyositis (JDM), Amyotophic lateral sclerosis

In [8]:
#LOOKING AT DATA SHAPE

# Get the expression data for GSE3307
expression_data_3307 = GSE3307.pivot_samples('VALUE')

# Get the expression data for GSE6011
expression_data_6011 = GSE6011.pivot_samples('VALUE')

# Now you can check the shape of the expression data
print("Dataset Number Rows,Columns")
print("GSE 3307 has:", expression_data_3307.shape)
print("GSE 6011 has:", expression_data_6011.shape)

Dataset Number Rows,Columns
GSE 3307 has: (44760, 242)
GSE 6011 has: (22283, 37)


In [9]:
#checking each gse object to insure we can extract the matrice properly

gse_object_to_inspect = GSE3307
# Get any GSM (sample) object. Here we take the first one
gsm = next(iter(gse_object_to_inspect.gsms.values()))
certainGSE1="GSE3307"
print(f"For GSE {certainGSE1}:", gsm.table.columns)


gse_object_to_inspect2 = GSE6011
# Get any GSM (sample) object. Here we take the first one
certainGSE1="GSE6011"
gsm = next(iter(gse_object_to_inspect2.gsms.values()))
print(f"For GSE {certainGSE1}:", gsm.table.columns)

#Output means we're good and we can make a matrix using ID_REF and Value

For GSE GSE3307: Index(['ID_REF', 'VALUE', 'ABS_CALL'], dtype='object')
For GSE GSE6011: Index(['ID_REF', 'VALUE'], dtype='object')


In [10]:
#Extracting Expression Matrices
#Goal : Getting the expression matrices ( essentially matrices with the information)
#Each row is a specific gene/genetic feature
#Each column is an individual sample
#Each value represents the gene expression in some regards.

def extract_expression_matrix(gse_object):
    """
    Extracts the expression matrix from a GSE object.
    An expression matrix is a matrix where each row represents a gene/genetic feature and each column represents an individual sample.

    Arguments:
        GSE_object: A GSE object from GEOparse ( essentially from the last code bubble )

    Returns:
        A pandas DataFrame representing the expression matrix. --> this is what we want in order to begin data pre-processing
    """
  # Initialize an empty dictionary to store expression data ( used later for indexing and formatting )
    expression_data = {}

    # Iterate over all GSM objects (samples)
    for gsm_id, gsm in gse_object.gsms.items():
        # Attempt to identify the correct column names for gene IDs and expression values
        id_col = None
        value_col = None
        for col in gsm.table.columns:
            if col.lower() in ['id', 'id_ref', 'probe_id', 'gene_id', 'gene_symbol']:
                id_col = col
            if col.lower() in ['value', 'expression', 'intensity']:
                value_col = col
        #code was used in the case of no id label in certain dataset - should be useless know, but keeping in case.
        if id_col is None or value_col is None:
            print(f"Warning: Could not identify ID or VALUE columns for sample {gsm_id}. Skipping.")
            continue  # Skip this sample if necessary columns are not found

        # Extract expression values and use gene IDs as index
        expression_data[gsm_id] = gsm.table.set_index(id_col)[value_col]

    # Create a pandas DataFrame from the expression data
    return pd.DataFrame(expression_data)

# Extract expression matrices

expression_matrix_gse3307 = extract_expression_matrix(GSE3307)
expression_matrix_gse6011 = extract_expression_matrix(GSE6011)
'''
"GSE3307"
"GSE6011"
'''
#gsms works pretty much by acting as an attribute to the GSE
#gsms pretty much acts as a dictionary where the IDs are keys and then each sample is a value in the dataset
#accessing specific datasets
num_samples_3307 = len(GSE3307.gsms) #gets dictionary for the 3307; you can only use the gsms attribute for GSEs
print(f"Number of samples in GSE3307: {num_samples_3307}")

num_samples_6011 = len(GSE6011.gsms) #gets dictionary for the 6011; you can only use the gsms attribute for GSEs
print(f"Number of samples in GSE6011: {num_samples_6011}")

Number of samples in GSE3307: 242
Number of samples in GSE6011: 37


In [11]:
#Expression Matrices
print(expression_matrix_gse3307.head(5))
print(expression_matrix_gse6011.head(5))

              GSM74239     GSM74240     GSM74241     GSM74242     GSM74243  \
ID_REF                                                                       
1007_s_at  1565.090088  1793.823975  2058.331543  2189.998047  1900.703247   
1053_at      53.745251   398.217529   476.876740   285.281464   418.731476   
117_at      483.366943   318.695801   233.067993   377.694519   336.891205   
121_at     3044.492676  3614.121826  2816.378418  2772.384033  3105.177002   
1255_g_at   207.656006   152.559189    99.692139   133.170013   176.241089   

              GSM74244     GSM74245     GSM74246     GSM74247     GSM74248  \
ID_REF                                                                       
1007_s_at  1986.621216  2423.252197  3030.981689  2691.871582  2191.285400   
1053_at     308.545441   205.691345   400.727661   186.328323   309.432556   
117_at      305.663177   277.269165   322.180786   300.988678   351.027740   
121_at     3346.971680  2559.171875  3803.663330  3449.002930  

In [12]:
#Verifying Similar Genes from Each DataSet

# Get the set of gene IDs from each matrix
genes_gse3307 = set(expression_matrix_gse3307.index)
genes_gse6011 = set(expression_matrix_gse6011.index)

# Check if the sets are equal
if genes_gse3307 == genes_gse6011:
    print("Gene sets are identical.")
else:
    print("Gene sets differ.")

    # Find the differences
    different_genes = genes_gse3307.symmetric_difference(genes_gse6011)
    print("Different genes:", different_genes)

# Assuming you have already calculated the sets of genes:
# genes_gse3307
# genes_gse6011

# Find common genes
common_genes = genes_gse3307.intersection(genes_gse6011)
print("Number of common genes:", len(common_genes))

# Calculate the percentage of overlap
overlap_percentage_3307 = len(common_genes) / len(genes_gse3307) * 100
overlap_percentage_6011 = len(common_genes) / len(genes_gse6011) * 100
print(f"Overlap percentage (GSE3307): {overlap_percentage_3307:.2f}%")
print(f"Overlap percentage (GSE6011): {overlap_percentage_6011:.2f}%")



Gene sets differ.
Different genes: {'228779_at', '226194_at', '243919_at', '230435_at', '237928_at', '243110_x_at', '237597_at', '222509_s_at', '222952_s_at', '236520_at', '231428_at', '227315_x_at', '240430_at', '239185_at', '225599_s_at', '225196_s_at', '226070_at', '224505_s_at', '224745_x_at', '229714_at', '244264_at', '237771_s_at', '236631_at', '235386_at', '242866_x_at', '224712_x_at', '241386_at', '232166_at', '237142_at', '233747_at', '231181_at', '229881_at', '234306_s_at', '227402_s_at', '244584_at', '230470_at', '228479_at', '242097_at', '223759_s_at', '226874_at', '228640_at', '232782_at', '226036_x_at', '225128_at', '224237_at', '230113_at', '237335_at', '233918_at', '225664_at', '230083_at', '242220_at', '235389_at', '240043_at', '224527_at', '232037_at', '228455_at', '244577_at', '232743_at', '225991_at', '230319_at', '236881_at', '226255_at', '222455_s_at', '229613_at', '228012_at', '242947_at', '232096_x_at', '225097_at', '228110_x_at', '223978_s_at', '238937_at', '23

So, this is kinda bad right off the dome, since now, we're finding that the genes differ extremely, so we're going to try to just cut the genes that don't match, since there is such a high quantity of genes.

In [13]:
# Filter out common genes, keeping only the first occurrence of each gene
common_genes = expression_matrix_gse3307.index.intersection(expression_matrix_gse6011.index)
data_3307 = expression_matrix_gse3307.loc[common_genes].reset_index(drop=True)  # Reset index and drop the old one
data_6011 = expression_matrix_gse6011.loc[common_genes].reset_index(drop=True)  # Reset index and drop the old one

print("Shape of filtered GSE3307:", data_3307.shape)
print("Shape of filtered GSE6011:", data_6011.shape)

Shape of filtered GSE3307: (22283, 242)
Shape of filtered GSE6011: (22283, 37)


In the short time that we've been looking at genetic overlap and differences, this happened : Number of samples in GSE3307: 242
Number of samples in GSE6011: 37
Somehow, there's an additional sample in each of them. Likely caused by some duplicate or another bug

In [14]:
#Let's problem Solve

print(data_3307.columns)

Index(['GSM74239', 'GSM74240', 'GSM74241', 'GSM74242', 'GSM74243', 'GSM74244',
       'GSM74245', 'GSM74246', 'GSM74247', 'GSM74248',
       ...
       'GSM121406', 'GSM121407', 'GSM121408', 'GSM121409', 'GSM121410',
       'GSM121412', 'GSM121413', 'GSM121414', 'GSM121415', 'GSM121416'],
      dtype='object', length=242)


In [16]:
#Removing extra columns

# Remove the 'dataset' column from both DataFrames
data_3307 = data_3307.drop('dataset', axis=1)
data_6011 = data_6011.drop('dataset', axis=1)

# Verify the changes
print("Columns in data_3307:", data_3307.columns)
print("Shape of data_3307:", data_3307.shape)

print("\nColumns in data_6011:", data_6011.columns)
print("Shape of data_6011:", data_6011.shape)

KeyError: "['dataset'] not found in axis"

Alr, so as you can see there's a final thing called "dataset" that should NOT be there, so all we gotta do is remove it

In [17]:
#Adding Conditions (Healthy or Impaired/Impacted) : 6011

#Conditions for 6011
num_samples_6011 = len(data_6011.columns)
conditions_6011 = ['Duchenne', 'Normal']

# Repeat the conditions list to match the number of samples, handling potential remainder
repeated_conditions_6011 = conditions_6011 * (num_samples_6011 // len(conditions_6011)) + conditions_6011[:num_samples_6011 % len(conditions_6011)]

#Aall this is doing is ensuring that we're going through the sample and assigning the conditions do it
if len(repeated_conditions_6011) == num_samples_6011:
    print("The length of repeated_conditions matches num_samples_6011.")
else:
    print("Lengths do not match! Something might be wrong.")
    print(f"Length of repeated_conditions: {len(repeated_conditions_6011)}")
    print(f"Value of num_samples_3307: {num_samples_6011}")

metadata_gse6011 = pd.DataFrame({
    'sample_id': data_6011.columns,
    'condition': repeated_conditions_6011
})


The length of repeated_conditions matches num_samples_6011.


In [20]:
#Adding Conditions (Healthy or Impaired/Impacted) : 3307

# Conditions for 3307
num_samples_3307 = len(data_3307.columns)
conditions_3307 = ['Normal', 'AQM', 'JDM', 'ALS', 'SPG4', 'FSHD', 'EDMD_XR', 'EDMD_AD', 'Becker', 'Duchenne', 'Calpain3', 'Dysferlin', 'FKRP']

# Repeat the conditions list to match the number of samples, handling potential remainder
repeated_conditions_3307 = conditions_3307 * (num_samples_3307 // len(conditions_3307)) + conditions_3307[:num_samples_3307 % len(conditions_3307)]

#Aall this is doing is ensuring that we're going through the sample and assigning the conditions do it
if len(repeated_conditions_3307) == num_samples_3307:
    print("The length of repeated_conditions matches num_samples_3307.")
else:
    print("Lengths do not match! Something might be wrong.")
    print(f"Length of repeated_conditions: {len(repeated_conditions_3307)}")
    print(f"Value of num_samples_3307: {num_samples_3307}")

metadata_gse3307 = pd.DataFrame({
    'sample_id': data_3307.columns,
    'condition': repeated_conditions_3307
})

The length of repeated_conditions matches num_samples_3307.


In [18]:
#TOTAL OCCURENCES

from collections import Counter

# Count the occurrences of each condition
condition_counts = Counter(repeated_conditions_3307)

# Print all conditions and their prevalence
print("3307")
for condition, count in condition_counts.items():
    print(f"{condition}: {count}")

# Count the occurrences of each condition
condition_counts = Counter(repeated_conditions_6011)

# Print all conditions and their prevalence
print("6011")
for condition, count in condition_counts.items():
    print(f"{condition}: {count}")



NameError: name 'repeated_conditions_3307' is not defined

In [19]:
#Normalization and Log Transformation

#function for preprocessing both at same time

def preprocess_data(expression_matrix):
    expression_matrix = np.log2(expression_matrix + 1)
    Q1 = expression_matrix.quantile(0.25)
    Q3 = expression_matrix.quantile(0.75)
    IQR = Q3 - Q1
    filtered_expression_matrix = expression_matrix[~((expression_matrix < (Q1 - 1.5 * IQR)) | (expression_matrix > (Q3 + 1.5 * IQR))).any(axis=1)]
    imputer = SimpleImputer(strategy='median')
    return pd.DataFrame(imputer.fit_transform(filtered_expression_matrix), columns=filtered_expression_matrix.columns)

# Apply preprocessing
data_3307 = preprocess_data(data_3307)
data_6011 = preprocess_data(data_6011)
#applying the prior function to logistically transform AND impute the data ( removing outliers )

# Combine datasets
combined_data = pd.concat([data_3307,data_6011], axis=1)
#axis is important for vertical concatenation



In [20]:
#Combined Data w/ Length
num_rows, num_cols = combined_data.shape
print(f"Number of rows in combined_data: {num_rows}")
print(f"Number of columns in combined_data: {num_cols}")
#Since number of columns is 279, we know we're on the right track
combined_data.head()
#So, if you take a look rows and columns represent
#Each Row ( Horizontal ) is a specific gene or genetic feature from dataset
#Each column represents an individual sample

Number of rows in combined_data: 21908
Number of columns in combined_data: 279


,GSM74239,GSM74240,GSM74241,GSM74242,GSM74243,GSM74244,GSM74245,GSM74246,GSM74247,GSM74248,...,GSM139528,GSM139529,GSM139530,GSM139531,GSM139532,GSM139533,GSM139534,GSM139535,GSM139536,GSM139537
0,10.612951,10.809627,11.007960,11.097372,10.893076,10.956827,11.243324,11.566045,11.394930,11.098220,...,3.374044,3.338871,3.321678,3.343624,3.338595,3.360426,3.318377,3.354667,3.368642,3.360756
1,5.774662,8.641031,8.900495,8.161290,8.713323,8.274007,7.691334,8.650074,7.549425,8.278136,...,2.842101,2.779439,2.759588,2.832192,2.783947,2.797841,2.775415,2.850664,2.805365,2.810597
2,8.919957,8.320556,7.870784,8.564891,8.400415,8.260511,8.120337,8.336198,8.238351,8.459545,...,2.977066,2.971662,2.993039,2.977600,2.977386,2.989434,2.965777,3.009890,2.996420,2.968307
3,11.572460,11.819829,11.460138,11.437432,11.600924,11.709072,11.322025,11.893553,11.752382,11.862890,...,3.456663,3.443225,3.463261,3.436054,3.407216,3.445067,3.438774,3.436627,3.505749,3.470584
4,7.704983,7.262651,6.653807,7.067918,7.469569,6.882175,7.288858,7.002675,6.634902,7.552593,...,2.629997,2.604865,2.638014,2.619648,2.589938,2.651910,2.610770,2.609192,2.674375,2.690348


In [22]:
#Honestly idk how this works, but I think it got me my much needed condition column so Ws in the shat

# Metadata for GSE3307
num_samples_3307 = len(data_3307.columns)
conditions = ['Normal', 'AQM', 'JDM', 'ALS', 'SPG4', 'FSHD', 'EDMD_XR', 'EDMD_AD', 'Becker', 'Duchenne', 'Calpain3', 'Dysferlin', 'FKRP']
# Repeat the conditions list to match the number of samples, handling potential remainder
repeated_conditions = conditions * (num_samples_3307 // len(conditions)) + conditions[:num_samples_3307 % len(conditions)]
#Aall this is doing is ensuring that we're going through the sample and assigning the conditions do it
if len(repeated_conditions) == num_samples_3307:
    print("The length of repeated_conditions matches num_samples_3307.")
else:
    print("Lengths do not match! Something might be wrong.")
    print(f"Length of repeated_conditions: {len(repeated_conditions)}")
    print(f"Value of num_samples_3307: {num_samples_3307}")


metadata_gse3307 = pd.DataFrame({
    'sample_id': data_3307.columns,
    'condition': repeated_conditions
})

# Metadata for GSE6011
metadata_gse6011 = pd.DataFrame({
    'sample_id': data_6011.columns,
    'condition': ['Duchenne'] * 22 + ['Normal'] * 15 # Changed 14 to 15 to match the number of samples
})

# Merge metadata
metadata = pd.concat([metadata_gse3307, metadata_gse6011]).reset_index(drop=True)

# Ensure all sample IDs match the columns in your combined data
# **Change: Reset index before joining to keep 'condition' as a column**
metadata = metadata.reset_index(drop=True)
combined_data_with_conditions = combined_data.T.join(metadata)

# **Change: No need to transpose again, 'condition' is already a column**
#combined_data_with_conditions = combined_data_with_conditions.T

# Update "Normal" condition to be the intersection of both datasets
normal_samples = metadata[metadata['condition'] == 'Normal']['sample_id'] # Extract sample IDs for Normal
combined_data_with_conditions['condition'] = combined_data_with_conditions.apply(lambda row: 'Normal' if row.name in normal_samples.values else row['condition'], axis=1)
# Apply to rows, checking if the sample ID (row.name) is in the normal samples

# List of all the disease conditions
disease_conditions = ['AQM', 'JDM', 'ALS', 'SPG4', 'FSHD', 'EDMD_XR', 'EDMD_AD', 'Becker', 'Duchenne', 'Calpain3', 'Dysferlin', 'FKRP']

# Iterate through the DataFrame and update 'condition'
for index, row in combined_data_with_conditions.iterrows():
    # Check if condition is not NaN and is in the disease list
    if pd.notna(row['condition']) and row['condition'] in disease_conditions:
        combined_data_with_conditions.at[index, 'condition'] = 'Affected'
    # If condition is NaN, assume it's affected
    elif pd.isna(row['condition']):
        combined_data_with_conditions.at[index, 'condition'] = 'Affected'

print(combined_data_with_conditions.head())

The length of repeated_conditions matches num_samples_3307.
                  0         1         2          3         4          5  \
GSM74239  10.612951  5.774662  8.919957  11.572460  7.704983  10.046811   
GSM74240  10.809627  8.641031  8.320556  11.819829  7.262651   9.758286   
GSM74241  11.007960  8.900495  7.870784  11.460138  6.653807   9.355207   
GSM74242  11.097372  8.161290  8.564891  11.437432  7.067918   9.402881   
GSM74243  10.893076  8.713323  8.400415  11.600924  7.469569   9.455832   

                 6         7         8         9  ...  21900  21901  21902  \
GSM74239  8.471959  8.272858  7.294602  6.261938  ...    NaN    NaN    NaN   
GSM74240  8.977301  6.729777  5.633661  7.117275  ...    NaN    NaN    NaN   
GSM74241  8.139373  7.981616  5.565327  7.680318  ...    NaN    NaN    NaN   
GSM74242  8.520008  6.761358  6.933544  7.531400  ...    NaN    NaN    NaN   
GSM74243  8.396848  5.884399  7.315637  6.738665  ...    NaN    NaN    NaN   

          21903  219

In [24]:
#Created "Affected" and "Normal"
num_nan_conditions = combined_data_with_conditions['condition'].isna().sum()
print(f"Number of NaN conditions: {num_nan_conditions}")

condition_counts = combined_data_with_conditions['condition'].value_counts()
print(condition_counts)

Number of NaN conditions: 0
condition
Affected    245
Normal       34
Name: count, dtype: int64


In [23]:
import statsmodels.api as sm

# Preparing for OLS Regression
expr_data = combined_data.T
expr_data['condition'] = combined_data_with_conditions['condition']  # Add the condition column

# Convert 'condition' column to numeric (0 for 'Normal', 1 for 'Affected')
expr_data['condition'] = expr_data['condition'].map({'Normal': 0, 'Affected': 1})

import statsmodels.api as sm
import pandas as pd

# ... (your existing code to prepare expr_data)

# Simplify the loop and handle potential errors
results_list = []
for gene in expr_data.columns[:-1]:
    try:
        # Prepare data, handling potential issues directly in the model fitting
        X = sm.add_constant(expr_data[gene])
        y = expr_data['condition']
        model = sm.OLS(y, X, missing='drop').fit()  # Drop rows with missing values
        results_list.append({
            'gene': gene,
            'pvalue': model.pvalues[gene],
            'coefficient': model.params[gene]
        })
    except Exception as e:
        print(f"Error processing gene {gene}: {e}")

# Print the results
for result in results_list:
    print(f"Gene: {result['gene']}, p-value: {result['pvalue']:.3f}, Coefficient: {result['coefficient']:.3f}")

Streaming output truncated to the last 5000 lines.
Gene: 16908, p-value: 0.000, Coefficient: 0.022
Gene: 16909, p-value: 0.000, Coefficient: 0.032
Gene: 16910, p-value: 0.000, Coefficient: 0.033
Gene: 16911, p-value: 0.000, Coefficient: 0.020
Gene: 16912, p-value: 0.000, Coefficient: 0.028
Gene: 16913, p-value: 0.000, Coefficient: 0.035
Gene: 16914, p-value: 0.000, Coefficient: 0.033
Gene: 16915, p-value: 0.000, Coefficient: 0.021
Gene: 16916, p-value: 0.000, Coefficient: 0.020
Gene: 16917, p-value: 0.000, Coefficient: 0.019
Gene: 16918, p-value: 0.003, Coefficient: 0.017
Gene: 16919, p-value: 0.000, Coefficient: 0.033
Gene: 16920, p-value: 0.000, Coefficient: 0.028
Gene: 16921, p-value: 0.000, Coefficient: 0.031
Gene: 16922, p-value: 0.000, Coefficient: 0.035
Gene: 16923, p-value: 0.001, Coefficient: 0.020
Gene: 16924, p-value: 0.000, Coefficient: 0.030
Gene: 16925, p-value: 0.000, Coefficient: 0.025
Gene: 16926, p-value: 0.000, Coefficient: 0.035
Gene: 16927, p-value: 0.000, Coeffici

In [26]:
#Normal and Affected
normal_data = combined_data_with_conditions[combined_data_with_conditions['condition'] == 'Normal'].drop('condition', axis=1)
affected_data = combined_data_with_conditions[combined_data_with_conditions['condition'] == 'Affected'].drop('condition', axis=1)

# Perform t-tests for each gene, ensuring numeric data
results = []
for gene in normal_data.columns:
    try:
        # Convert data to numeric, handling potential errors
        normal_values = pd.to_numeric(normal_data[gene], errors='coerce')
        affected_values = pd.to_numeric(affected_data[gene], errors='coerce')

        statistic, pvalue = ttest_ind(normal_values, affected_values, nan_policy='omit')
        results.append({'gene': gene, 'pvalue': pvalue})
    except Exception as e:
        print(f"Error processing gene {gene}: {e}")

# Create a DataFrame from results and sort by p-value
results_df = pd.DataFrame(results).sort_values('pvalue')

# Select top k features (e.g., k=10)
top_k_features = results_df.head(15)['gene'].tolist()

print(top_k_features)

[2259, 19471, 15894, 20481, 636, 34, 12206, 13227, 8088, 11086, 386, 15889, 36, 11244, 11667]


In [31]:
# Select the expression data for the top k features
X = combined_data_with_conditions[top_k_features]
y = combined_data_with_conditions['condition']

# Now X will have the shape (279, 10) matching the number of samples in y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, pos_label='Affected'), # Change 'MD' to 'Affected'
        'Recall': recall_score(y_test, y_pred, pos_label='Affected'), # Change 'MD' to 'Affected'
        'F1 Score': f1_score(y_test, y_pred, pos_label='Affected') # Change 'MD' to 'Affected'
    #Accuracy - how many that were normal were normal and were affected were affected
    #Precision - True Positives/Total Positives
    #Recall - True Positives/True Positive + False Negatives (Out of all that were affected, how many did it get)
    #F1 Score - harmonic mean of Precision and Recall

        }


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create the Logistic Regression model
lr_model = LogisticRegression(penalty='l2', C=0.1, max_iter=1000)

# Now use X_train_scaled and X_test_scaled in your model evaluation
lr_metrics = evaluate_model(lr_model, X_train_scaled, X_test_scaled, y_train, y_test)



nb_model = GaussianNB()
nb_metrics = evaluate_model(nb_model, X_train, X_test, y_train, y_test)

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_metrics = evaluate_model(knn_model, X_train, X_test, y_train, y_test)

rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_metrics = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

print("Logistic Regression:", lr_metrics)
print("Naive Bayes:", nb_metrics)
print("k-Nearest Neighbors:", knn_metrics)
print("Random Forest:", rf_metrics)


Logistic Regression: {'Accuracy': 0.9107142857142857, 'Precision': 0.9107142857142857, 'Recall': 1.0, 'F1 Score': 0.9532710280373832}
Naive Bayes: {'Accuracy': 0.8571428571428571, 'Precision': 0.9387755102040817, 'Recall': 0.9019607843137255, 'F1 Score': 0.92}
k-Nearest Neighbors: {'Accuracy': 0.8571428571428571, 'Precision': 0.9215686274509803, 'Recall': 0.9215686274509803, 'F1 Score': 0.9215686274509803}
Random Forest: {'Accuracy': 0.9107142857142857, 'Precision': 0.9423076923076923, 'Recall': 0.9607843137254902, 'F1 Score': 0.9514563106796117}


In [164]:
!pip install GEOparse
import GEOparse

print(type(GSE3307))

print(GSE3307.gsms.keys())

<class 'GEOparse.GEOTypes.GSE'>
dict_keys(['GSM74239', 'GSM74240', 'GSM74241', 'GSM74242', 'GSM74243', 'GSM74244', 'GSM74245', 'GSM74246', 'GSM74247', 'GSM74248', 'GSM74351', 'GSM74352', 'GSM74353', 'GSM74354', 'GSM74355', 'GSM74356', 'GSM74357', 'GSM74358', 'GSM74359', 'GSM74360', 'GSM74361', 'GSM74362', 'GSM74363', 'GSM74364', 'GSM74365', 'GSM74366', 'GSM74367', 'GSM74368', 'GSM74369', 'GSM74370', 'GSM74371', 'GSM74372', 'GSM74373', 'GSM74374', 'GSM74375', 'GSM74376', 'GSM74377', 'GSM74378', 'GSM74379', 'GSM74380', 'GSM74381', 'GSM74382', 'GSM74383', 'GSM74384', 'GSM74385', 'GSM74386', 'GSM74387', 'GSM74388', 'GSM74389', 'GSM74390', 'GSM74391', 'GSM74392', 'GSM74393', 'GSM74394', 'GSM74395', 'GSM74396', 'GSM74397', 'GSM74398', 'GSM74399', 'GSM74400', 'GSM74401', 'GSM74402', 'GSM74403', 'GSM74404', 'GSM74405', 'GSM74406', 'GSM74407', 'GSM74408', 'GSM74409', 'GSM74410', 'GSM74411', 'GSM74412', 'GSM74413', 'GSM74414', 'GSM74415', 'GSM74416', 'GSM74417', 'GSM74418', 'GSM74419', 'GSM11993

In [2]:
!pip install GEOparse
import GEOparse

# Fetch and parse the GSE3307 dataset
GSE3307 = GEOparse.get_GEO(geo="GSE3307", destdir="./")

# Now continue with your code to process GSE3307
for key in GSE3307.keys():
    if isinstance(GSE3307[key], GEOparse.gpl.GPL):
        print(f"Key: {key}, Contains GPL Object: True")
        # Accessing gene information from GPL object (example)
        for entry in GSE3307[key].table.itertuples():
            print(entry.ID, entry.Gene_Symbol)  # Adjust column names as needed
        break  # Stop after finding the GPL object
    else:
        print(f"Key: {key}, Contains GPL Object: False")

13-Jul-2024 22:57:40 DEBUG utils - Directory ./ already exists. Skipping.
DEBUG:GEOparse:Directory ./ already exists. Skipping.
13-Jul-2024 22:57:40 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE3307_family.soft.gz to ./GSE3307_family.soft.gz
INFO:GEOparse:Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE3307_family.soft.gz to ./GSE3307_family.soft.gz
100%|██████████| 62.8M/62.8M [00:01<00:00, 40.6MB/s]
13-Jul-2024 22:57:42 DEBUG downloader - Size validation passed
DEBUG:GEOparse:Size validation passed
13-Jul-2024 22:57:42 DEBUG downloader - Moving /tmp/tmpc6xcnagd to /content/GSE3307_family.soft.gz
DEBUG:GEOparse:Moving /tmp/tmpc6xcnagd to /content/GSE3307_family.soft.gz
13-Jul-2024 22:57:42 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE3307_family.soft.gz
DEBUG:GEOparse:Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE3nnn/GSE3307/soft/GSE330

AttributeError: 'GSE' object has no attribute 'keys'